In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import skorch
import torch
from skorch import NeuralNetClassifier
from tqdm import tqdm
from torchvision import transforms
import torch.nn as nn
from scipy.stats import entropy
import random
import h5py
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import FeatureAgglomeration

In [2]:


# Compare Algorithmsa
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [3]:
from sklearn.metrics import plot_confusion_matrix, matthews_corrcoef, classification_report,confusion_matrix, accuracy_score, balanced_accuracy_score, cohen_kappa_score, f1_score,  precision_score, recall_score
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay


def classification_complete_report(y_true, y_pred ,labels = None  ): 
    print(classification_report(y_true, y_pred, labels = labels))
    print(15*"----")
    print("matthews correlation coeff: %.4f" % (matthews_corrcoef(y_true, y_pred)) )
    print("Cohen Kappa score: %.4f" % (cohen_kappa_score(y_true, y_pred)) )
    print("Accuracy: %.4f & balanced Accuracy: %.4f" % (accuracy_score(y_true, y_pred), balanced_accuracy_score(y_true, y_pred)) )
    print("macro F1 score: %.4f & micro F1 score: %.4f" % (f1_score(y_true, y_pred, average = "macro"), f1_score(y_true, y_pred, average = "micro")) )
    print("macro Precision score: %.4f & micro Precision score: %.4f" % (precision_score(y_true, y_pred, average = "macro"), precision_score(y_true, y_pred, average = "micro")) )
    print("macro Recall score: %.4f & micro Recall score: %.4f" % (recall_score(y_true, y_pred, average = "macro"), recall_score(y_true, y_pred, average = "micro")) )
    cm = confusion_matrix(y_true, y_pred,labels= labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    print(15*"----")

In [4]:
metadata = pd.read_csv("/projects/site/pred/data_science/year/2022/immunological_synapse/synapse_data_features/metadata_subset_with_predictions.csv.gz")
metadata

df_features = pd.read_csv("/projects/site/pred/data_science/year/2022/immunological_synapse/synapse_data_features/df_features_subset.csv.gz")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
metadata["key"] = metadata["experiment"] + "_" + metadata["donor"] 

In [6]:
set_of_interesting_classes = ['B_cell',  'T_cell', 
                        'T_cell_with_signaling',
                        'T_cell_with_B_cell_fragments',
                        'B_T_cell_in_one_layer',
                        'Synapses_without_signaling', 
                        'Synapses_with_signaling',
                        'No_cell_cell_interaction', 
                        'Multiplets']

In [7]:


label_map = dict()
for i, cl in enumerate(set_of_interesting_classes):
    label_map[cl] = i
label_map['-1'] = -1
label_map[-1] = -1

inv_map = {v: k for k, v in label_map.items()}

In [8]:
inv_map


{0: 'B_cell',
 1: 'T_cell',
 2: 'T_cell_with_signaling',
 3: 'T_cell_with_B_cell_fragments',
 4: 'B_T_cell_in_one_layer',
 5: 'Synapses_without_signaling',
 6: 'Synapses_with_signaling',
 7: 'No_cell_cell_interaction',
 8: 'Multiplets',
 -1: -1}

In [9]:
labeled_index = metadata["label"].isin(set_of_interesting_classes)

train_index = metadata["set"] == "train"
train_index = train_index[train_index].index

validation_index = metadata["set"] == "validation"
validation_index = validation_index[validation_index].index

test_index = metadata["set"] == "test"
test_index = test_index[test_index].index

## Donor cross validation & experiment cross validation

In [10]:
list_of_features_to_exclude = ["Antibody","Live-Dead","CD18",  "_R2",  "_R3",  "_R8", "background",
                               "moment","percent","mask_based_mean","mask_based_min","mask_based_max",  
                               "RMS","intensity_BF","_R1_R2","_R1_R3","_R1_R4","_R1_R5","_R1_R6","_R1_R7",
                               "_R1_R8","_R2_R1","_R2_R3","_R2_R4","_R2_R5","_R2_R6","_R2_R7","_R2_R8",
                               "_R3_R1","_R3_R2","_R3_R4","_R3_R5","_R3_R6","_R3_R7","_R3_R8",
                               "_R4_R1","_R4_R2","_R4_R3","_R4_R5","_R4_R6","_R4_R7","_R4_R8",
                               "_R5_R1","_R5_R2","_R5_R3","_R5_R4","_R5_R8","_R6_R1","_R6_R2",
                               "_R6_R3","_R6_R4","_R6_R5","_R6_R7","_R6_R8","_R7_R1","_R7_R2",
                               "_R7_R3","_R7_R4","_R7_R5","_R7_R6","_R7_R8","_R8_R1","_R8_R2",
                               "_R8_R3","_R8_R4","_R8_R5","_R8_R6","_R8_R7","cell_distance"]
classification_indx = ~df_features.columns.str.contains( list_of_features_to_exclude[0]) 

for f_to_exclude in list_of_features_to_exclude:
    classification_indx = classification_indx & ~df_features.columns.str.contains( f_to_exclude) 
    

print("list of remained columns:\n", 
      df_features.columns[classification_indx],
     "\n",
     "so we have these many columns:",
     classification_indx.sum())

list of remained columns:
 Index(['mask_based_area_BF', 'mask_based_bbox_area_BF',
       'mask_based_convex_area_BF', 'mask_based_eccentricity_BF',
       'mask_based_equivalent_diameter_BF', 'mask_based_euler_number_BF',
       'mask_based_extent_BF', 'mask_based_feret_diameter_max_BF',
       'mask_based_filled_area_BF', 'mask_based_major_axis_length_BF',
       ...
       'max_intensity_ratio_F-Actin_R5_R7', 'sum_intensity_ratio_MHCII_R5_R7',
       'mean_intensity_ratio_MHCII_R5_R7', 'max_intensity_ratio_MHCII_R5_R7',
       'sum_intensity_ratio_CD3_R5_R7', 'mean_intensity_ratio_CD3_R5_R7',
       'max_intensity_ratio_CD3_R5_R7', 'sum_intensity_ratio_P-CD3zeta_R5_R7',
       'mean_intensity_ratio_P-CD3zeta_R5_R7',
       'max_intensity_ratio_P-CD3zeta_R5_R7'],
      dtype='object', length=247) 
 so we have these many columns: 247


In [11]:
label_map = dict()
for i, cl in enumerate(set_of_interesting_classes):
    label_map[cl] = i

label_map['-1'] = -1
label_map[-1] = -1

print(label_map)

{'B_cell': 0, 'T_cell': 1, 'T_cell_with_signaling': 2, 'T_cell_with_B_cell_fragments': 3, 'B_T_cell_in_one_layer': 4, 'Synapses_without_signaling': 5, 'Synapses_with_signaling': 6, 'No_cell_cell_interaction': 7, 'Multiplets': 8, '-1': -1, -1: -1}


In [12]:

selected_feature_names = ['mask_based_major_axis_length_MHCII',
       'mask_based_major_axis_length_CD3', 'correlation_F-Actin',
       'contrast_P-CD3zeta', 'homogeneity_F-Actin', 'ASM_CD3',
       'mean_intensity_ratio_F-Actin_R5_R6',
       'max_intensity_ratio_MHCII_R5_R6', 'mask_based_solidity_F-Actin',
       'max_intensity_ratio_F-Actin_R5_R6',
       'mask_based_equivalent_diameter_BF', 'dissimilarity_F-Actin',
       'mask_based_eccentricity_CD3',
       'mean_intensity_ratio_P-CD3zeta_R5_R6', 'correlation_CD3',
       'manders_overlap_coefficient_R5_MHCII_R7_P-CD3zeta',
       'correlation_P-CD3zeta', 'mask_based_solidity_P-CD3zeta',
       'mean_intensity_P-CD3zeta', 'skew_intensity_P-CD3zeta',
       'correlation_distance_R5_MHCII_R6_CD3', 'dissimilarity_P-CD3zeta',
       'contrast_CD3', 'sum_intensity_CD3',
       'sum_intensity_ratio_MHCII_R5_R6', 'mask_based_eccentricity_BF',
       'homogeneity_BF', 'mask_based_solidity_CD3', 'mask_based_area_CD3',
       'kurtosis_intensity_P-CD3zeta', 'mean_intensity_CD3',
       'skew_intensity_MHCII', 'mask_based_feret_diameter_max_BF',
       'correlation_MHCII', 'mask_based_major_axis_length_P-CD3zeta',
       'mask_based_bbox_area_CD3', 'mean_intensity_ratio_CD3_R5_R6',
       'manders_overlap_coefficient_R5_MHCII_R6_CD3',
       'mean_intensity_ratio_MHCII_R5_R6',
       'mask_based_feret_diameter_max_CD3', 'kurtosis_intensity_CD3',
       'homogeneity_CD3', 'sum_intensity_ratio_CD3_R5_R6',
       'mask_based_eccentricity_F-Actin',
       'structural_similarity_R4_F-Actin_R5_MHCII',
       'max_intensity_ratio_P-CD3zeta_R5_R6', 'mask_based_solidity_MHCII',
       'mean_intensity_F-Actin', 'homogeneity_MHCII',
       'mean_intensity_MHCII', 'contrast_MHCII', 'dissimilarity_MHCII',
       'dissimilarity_CD3', 'skew_intensity_F-Actin',
       'mask_based_extent_BF', 'max_intensity_ratio_CD3_R5_R6',
       'mask_based_major_axis_length_F-Actin',
       'euclidean_distance_R5_MHCII_R6_CD3', 'std_intensity_P-CD3zeta',
       'mask_based_eccentricity_MHCII', 'dice_distance_BF_CD3',
       'intensity_correlation_quotient_R5_MHCII_R6_CD3',
       'mask_based_eccentricity_P-CD3zeta', 'skew_intensity_CD3',
       'sum_intensity_F-Actin', 'correlation_BF']